In [ ]:
import numpy as np
from geneticalgorithm import geneticalgorithm as ga
import pickle
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

from tensorflow.keras.preprocessing import image


from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import activations
from tqdm import tqdm

In [ ]:
non_image = pickle.load(open('forest.sav', 'rb'))
resnet = tf.keras.models.load_model('New_RES_Augmentated.h5', compile=False)
VGG = tf.keras.models.load_model('New_VGG_Augmented.h5', compile=False)

In [ ]:
df = pd.read_csv('./data/aug_HAM10000_Metadata')

df.head()

In [ ]:
df = df.sample(frac = 1).reset_index()

df.head()

In [ ]:
df['age'] = df['age'].fillna(0)
feature_vector = []
y = []

for i in tqdm(range(0, 10015)):
    sex = [0, 0]
    if df.iloc[i]['sex'] == 'male':
        sex = [1, 0]
    elif df.iloc[i]['sex'] == 'female':
        sex = [0, 1]
    
    age = df.iloc[i]['age']
    y.append(df.iloc[i]['risk'])
    loc = df.iloc[i]['localization']
    
    sex.append(age)
    feat = np.array(sex)
    
    if loc == 'abdomen':
        feat = np.concatenate((feat, tf.one_hot(0, 14)))
    elif loc == 'scalp':
        feat = np.concatenate((feat, tf.one_hot(1, 14)))
    elif loc == 'lower extremity':
        feat = np.concatenate((feat, tf.one_hot(2, 14)))
    elif loc == 'trunk':
        feat = np.concatenate((feat, tf.one_hot(3, 14)))
    elif loc == 'upper extremity':
        feat = np.concatenate((feat, tf.one_hot(4, 14)))
    elif loc == 'back':
        feat = np.concatenate((feat,tf.one_hot(5, 14)))
    elif loc == 'neck':
        feat = np.concatenate((feat,tf.one_hot(6, 14)))
    elif loc == 'face':
        feat = np.concatenate((feat,tf.one_hot(7, 14)))
    elif loc == 'chest':
        feat = np.concatenate((feat,tf.one_hot(8, 14)))
    elif loc == 'foot':
        feat = np.concatenate((feat,tf.one_hot(9, 14)))
    elif loc == 'ear':
        feat = np.concatenate((feat,tf.one_hot(10, 14)))
    elif loc == 'unknown':
        feat = np.concatenate((feat, np.zeros(14)))
    elif loc == 'hand':
        feat = np.concatenate((feat,tf.one_hot(11, 14)))
    elif loc == 'acral':
        feat = np.concatenate((feat,tf.one_hot(12, 14)))
    elif loc == 'genital':
        feat = np.concatenate((feat,tf.one_hot(13, 14)))

    feature_vector.append(feat)

feat_X = np.array(feature_vector)

In [ ]:
vgg_preds = []

VGG_load = VGG16(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(VGG_load)
model.add(layers.Flatten())


for i in tqdm(range(0, TOTAL)):
    file = df.iloc[i]['image_id']
    feat = feat_X[i]
    
    img = image.load_img('./data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    preds = model.predict(img).reshape(25088)
    
    vgg_preds.append(VGG.predict(preds.reshape(1, -1)))

In [ ]:
pickle.dump(vgg_preds, open('vgg_preds', 'wb'))

In [ ]:
res_preds = []

res_load = ResNet152V2(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(res_load)
model.add(layers.Flatten())


for i in tqdm(range(0, TOTAL)):
    file = df.iloc[i]['image_id']
    feat = feat_X[i]
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.resnet.preprocess_input(img)
    preds = model.predict(img).reshape(100352)
    
    res_preds.append(resnet.predict(preds.reshape(1, -1)))

In [ ]:
pickle.dump(vgg_preds, open('resnet_preds', 'wb'))

In [ ]:
non_preds = non_image.predict_proba(feat_X)

In [ ]:
pickle.dump(non_preds, open('non_preds', 'wb'))

In [ ]:
def fitness(X):
    p1 = X[0]
    p2 = X[1]
    p3 = 1 - X[0] - X[1]
    predict = p1 * vgg_preds + p2 * resnet_preds + p3 * non_preds
    
    cce = tf.keras.losses.CategoricalCrossentropy()
    return cce(y, predict)

In [ ]:
varbound=np.array([[0,0.5]]*2)

In [ ]:
algorithm_param = {'max_num_iteration': 3000,\
                   'population_size':100,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None}


model=ga(function=fitness,
         dimension=2,
         variable_type='real',
         variable_boundaries=varbound,
         algorithm_parameters=algorithm_param)

In [ ]:
model.run()

In [ ]:
model.output_dict